# Web Scraping: A Simple Application

Benjamin Wallace (May 2019)

This is an example application of web scraping. [OTTIAQ](https://ottiaq.org/en/) is an organization for translators in the Canadian province of Québec. Their website includes a [search engine](https://ottiaq.org/en/services-for-the-public-and-businesses/find-a-professional/) for finding a translator according to criteria including source language, target language, and region. In order to reduce bias towards translators whose last names come early in a lexicographic ordering, the results are returned in a random order.

We wish to test whether the returned results are indeed unbiased. To do this, we write a web scraper to repeatedly perform a search and aggregate the ranks of the returned results into an sqlite3 database. The rank distribution can then be test for uniformity using standard statistical methods ($\chi^2$ test).

## Ethics of web scraping

While web scrapers are typically not illegal, it's best to strive to follow some unwritten ethical guidelines when using them in order, for example, not to overload the server being scraped with requests.

A good place to start is with the site's [robots.txt](https://ottiaq.org/robots.txt). As of May 2019, this file contains no restrictions against scraping the site's search page.

![robots](robots.txt.png)

## Forming an HTTP request

HTTP requests in Python are easily formed using the third-party `requests` package (we could also use `urllib`, which is part of the Python standard library). We will also make use of the `re` package from the standard library for regular expressions to search through the resulting HTML response.

In [122]:
import requests
import re

For the purposes of this demonstration, we will consider the results of searching for translators with the default parameters in the first search box (French to English, no specified region).

In order to determine what kind of HTTP request to make (GET/POST/etc. and with with variable/value pairs), we need to inspect the search form element. This is easily done in chrome using the "Inspect Element" tool. For instance, below we see that the variable carrying the source language information is called `langue_depart_off`. Similarly, the target variable is `langue_arrivee_off` and the desired values in our case are `FR` and `EN`, respectively. Lastly, we find the form acts by POSTing data to `https://ottiaq.org/en/directory/results/`.

![Inspect element](inspect.png)

Let's save the information we've gathered into the following variables.

In [123]:
url = 'https://ottiaq.org/en/directory/results/'
data = {'langue_depart_off': 'FR', 'langue_arrivee_off': 'EN'}

By performing an actual search, we find that each returned result includes a convenient id specifier prefaced by the phrase "Membership number : ". This id is thus easily captured with the following regular expression.

In [124]:
pattern = re.compile("(?<=Membership number : )\d+")

Since we'll need to perform the HTTP request and then extract the id using the regex pattern above many times, we'll package this sequence of operations into the following function.

In [125]:
def scrape(url, data):
    response = requests.post(url, data)
    html = response.text
    matches = pattern.findall(html)
    
    return list(map(int, matches))

## Initializing a database

We'll store the results of our web scraping into an sqlite3 database, which we can interface with through the `sqlite3` package from the Python standard library.

In [126]:
import sqlite3

The pertinent information is the user id, and how many times the user has appeared in each possible rank. The number of possible ranks is simply the number of results returned by a single search query, so we begin by applying the function we defined above. We see that, for our chosen query parameters, there are 109 results.

In [127]:
matches = scrape(url, data)

In [128]:
num_results = len(matches)
num_results

109

Let's start by connecting to our database (the database is created if it doesn't exist yet).

In [129]:
conn = sqlite3.connect('ottiaq.db')
c = conn.cursor()

We'll create a table for our current search query parameters (French to English) called `fren`.

In [130]:
table = 'fren'

We could uncomment and run the following code if we wanted to start from scratch.

In [131]:
# optional: reset table
# query_drop = 'DROP TABLE IF EXISTS {};'.format(table)
# c.execute(query_drop)
# conn.commit()

Our table contains a column for the user id's and one for each possible rank (initialized to 0) for a total of `num_results + 1` matches.

In [132]:
# query_create = 'CREATE TABLE IF NOT EXISTS fren (id INTEGER PRIMARY KEY NOT NULL'
query_create = 'CREATE TABLE IF NOT EXISTS {} (id INTEGER PRIMARY KEY NOT NULL'.format(table)

for i in range(num_results):
    query_create += ',rank{} INTEGER DEFAULT 0'.format(i)

query_create += ');'

In [133]:
c.execute(query_create)
conn.commit()

Next, we insert the id's we found above into the first column of our table. Above, we specified the id as the primary key, which must be unique, so we specify that our `INSERT` query can be ignored (if the id already exists).

In [134]:
query_insert = 'INSERT OR IGNORE INTO {} (id) VALUES (?);'.format(table)
for i in range(num_results):
    c.execute(query_insert, (matches[i],))
conn.commit()

The rank columns' values are all 0 by default, but since we have already queried the search engine 1 time, each row should now have a single rank column set to 1. Later, when we perform repeated searches we will want to update the number of times each id came in each rank, so once again we package the desired sequence of operations into a function before performing it.

In [135]:
def update_ranks(matches):
    for rank, match in enumerate(matches):
        c.execute('SELECT rank{} FROM {} WHERE id=?'.format(rank, table), (match,))
        
        n = c.fetchone()[0] # fetchone returns a tuple
        c.execute('UPDATE {} SET rank{}=? WHERE id=?'.format(table, rank), (n + 1, match))
    conn.commit()

In [136]:
update_ranks(matches)

## Scraping for data

At this point, most of the work needed to scrape the date has been done. We simple need to iterate the `scrape` and `update_ranks` functions (later, we will discuss the number of times we should do this in order to ensure statistically significant results). Again, as a courtesy, we space out our HTTP requests by a certain wait time.

In [137]:
import time

In [138]:
def collect(num_requests, wait_time=0.5):
    for N in range(num_requests):
        time.sleep(wait_time)

        matches = scrape(url, data)
        update_ranks(matches)
        
        if (N % 50 == 0):
            print("Iteration {}".format(N))

Before running `collect`, we should decide how many requests to make. This will be discussed in the next section based on statistical considerations.

## Statistical analysis

Rather than testing for uniformity of the sampled permutations of members, which would be computationally expensive, we will test for uniformity of samples found within a given rank, say rank 0.

### Pearson's $\chi^2$ test

Consider a general situation in which we are sampling from a categorical distribution with $K$ categories. We wish to test the null hypothesis that category $i$ occurs with probability $p_i$ for $i = 1, \ldots, K$. If this is the case, then for a sample of size $N$, the expected number of observed occurences of category $i$ is $N p_i$. If $N_i$ is the actual number of observations from category $i$, then $(N_i - N p_i)^2$ should be small in order for the null hypothesis to be true. **Pearson's $\chi^2$ test** rejects the null hypothesis if the test statistic
$$\hat\chi^2 = \sum_{i=1}^K \frac{(N_i - N p_i)^2}{N p_i}$$
is sufficiently small.

What constitutes *sufficiently small*? This can be answered by a theorem of Karl Pearson, which states that $\hat\chi^2$ has an approximate $\chi^2$ distribution with $K - 1$ degrees of freedom. A very heuristic explanation involves writing $N_i = \sum_{j=1}^N \mathbf{1}(X_j = i)$ and applying the central limit theorem to see that, assuming that the null hypothesis is true, then $\frac{N_i - N p_i}{\sqrt{N p_i}}$ is approximately normally distributed with variance $(1 - p_i)$. Thus, $\hat\chi^2$ is a sum of squares of (non-independent) normal random variables, which should be contrasted with the fact that a $\chi^2$ random variable is a sum of squares of *independent standard normal* random variables.

The upshot is we can use the statistics of $\chi^2$ squared distribution to determine the maximum size of $\hat\chi^2$ for which we can assert the null hypothesis with significance level $\alpha = 0.05$.

The number of samples $n$ must also be sufficiently large for the test to be meaningful. The usual heuristic is that samples should be collected until (1) $N p_i \ge 1$ for each $i$ and (2) $n p_i \ge 5$ for at least $80\%$ of $i$. When testing for the uniform distribution, the second condition is stronger: we require that $N \ge 5 \times \mathrm{num\_results}$.

The total number of samples in our database can be found by summing along rank columns (the result should actually be independent of the column).

In [139]:
def get_ranks(rank):
    return list(map(lambda x: x[0], c.execute('SELECT rank{} FROM fren'.format(rank)).fetchall()))

def num_samples(rank):
    ranks = get_ranks(rank)
    return sum(ranks)

Let's gather twice as much as the minimum required by the heuristics.

In [140]:
rank = 0
min_samples = 5 * num_results

num_requests = max(0, 2 * min_samples - num_samples(rank))

In [141]:
collect(num_requests)

The hypothesis test can be performed using `scipy.stats`.

In [142]:
import scipy.stats as stats

In [143]:
ranks = get_ranks(rank)
stats.chisquare(ranks)

Power_divergenceResult(statistic=137.17323556370306, pvalue=0.030433347502135482)

In [144]:
ranks = get_ranks(rank)
stats.chisquare(ranks)

Power_divergenceResult(statistic=137.17323556370306, pvalue=0.030433347502135482)

This result indicates uniformity of the results in the top rank.